# Import packages

In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys

project_folder = '/Users/dusiyi/Documents/Multifirefly-Project'
os.chdir(project_folder)
sys.path.append(os.path.join(project_folder, 'multiff_analysis', 'methods'))

from data_wrangling import specific_utils, process_monkey_information
from pattern_discovery import pattern_by_trials, pattern_by_trials, cluster_analysis, organize_patterns_and_features
from visualization.matplotlib_tools import plot_behaviors_utils
from non_behavioral_analysis.neural_data_analysis.get_neural_data import neural_data_processing
from non_behavioral_analysis.neural_data_analysis.visualize_neural_data import plot_neural_data, plot_modeling_result
from non_behavioral_analysis.neural_data_analysis.model_neural_data import cca_class, pgam_class, neural_data_modeling, reduce_multicollinearity
from non_behavioral_analysis.neural_data_analysis.neural_vs_behavioral import prep_monkey_data, prep_target_data, neural_vs_behavioral_class
from non_behavioral_analysis.neural_data_analysis.planning_neural import planning_neural_class, planning_neural_utils

import sys
import math
import gc
import subprocess
from pathlib import Path
from importlib import reload

# Third-party imports
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rc
from scipy import linalg, interpolate
from scipy.signal import fftconvolve
from scipy.io import loadmat
from scipy import sparse
import torch
from numpy import pi

# Machine Learning imports
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.cross_decomposition import CCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.multivariate.cancorr import CanCorr

# Neuroscience specific imports
import neo
import rcca

plt.rcParams["animation.html"] = "html5"
os.environ['KMP_DUPLICATE_LIB_OK']='True'
rc('animation', html='jshtml')
matplotlib.rcParams.update(matplotlib.rcParamsDefault)
matplotlib.rcParams['animation.embed_limit'] = 2**128
pd.set_option('display.float_format', lambda x: '%.5f' % x)
np.set_printoptions(suppress=True)
print("done")

%load_ext autoreload
%autoreload 2

done
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
def supply_info_of_cluster_to_df(df, ff_real_position_sorted, ff_life_sorted, monkey_information, max_cluster_distance=50):

    ff_time = monkey_information.loc[df['point_index'].values, 'time'].values
    ff_cluster = cluster_analysis.find_alive_ff_clusters(ff_real_position_sorted[df['ff_index'].values], ff_real_position_sorted, ff_time-10, ff_time+10,
                                                         ff_life_sorted, max_distance=max_cluster_distance)
    ff_cluster_df = cluster_analysis.turn_list_of_ff_clusters_info_into_dataframe(
        ff_cluster, df['point_index'].values)
    # new_df = decision_making_utils.find_many_ff_info_anew(ff_cluster_df['ff_index'].values, ff_cluster_df['point_index'].values, ff_real_position_sorted, ff_dataframe_visible, monkey_information)
    return ff_cluster_df


In [ ]:
    ff_cluster = cluster_analysis.find_alive_ff_clusters(ff_real_position_sorted[df['ff_index'].values], ff_real_position_sorted, ff_time-10, ff_time+10,
                                                         ff_life_sorted, max_distance=max_cluster_distance)
    ff_cluster_df = cluster_analysis.turn_list_of_ff_clusters_info_into_dataframe(
        ff_cluster, df['point_index'].values)

In [ ]:
data_item.make_or_retrieve_target_clust_last_vis_df()

In [ ]:
## or target cluster_analysis

target_cluster_TAFT = target_clust_last_vis_df[target_clust_last_vis_df['target_index'].isin(try_a_few_times_trials)]

In [ ]:


def get_target_clust_last_vis_df(ff_dataframe, monkey_information, ff_caught_T_new, ff_real_position_sorted, ff_life_sorted, duration_of_evaluation=3,
                                 max_distance_to_target_in_cluster=50, keep_all_rows=False):
    ff_indices_of_each_cluster = find_alive_target_clusters(
        ff_real_position_sorted, ff_caught_T_new, ff_life_sorted, max_distance=max_distance_to_target_in_cluster)
    target_clust_last_vis_df = _get_target_clust_last_vis_df(ff_dataframe, monkey_information, ff_caught_T_new, ff_real_position_sorted,
                                                             max_distance_to_target_in_cluster=max_distance_to_target_in_cluster, duration_of_evaluation=duration_of_evaluation)
    target_clust_last_vis_df['nearby_alive_ff_indices'] = ff_indices_of_each_cluster
    if not keep_all_rows:
        # drop the rows whose last_vis_dist is 9999
        target_clust_last_vis_df = target_clust_last_vis_df[target_clust_last_vis_df['last_vis_dist'] != 9999].copy(
        )
        # also drop target_index = 0
        target_clust_last_vis_df = target_clust_last_vis_df[target_clust_last_vis_df['target_index'] != 0].copy(
        )
    else:
        target_clust_last_vis_df = target_clust_last_vis_df
    return target_clust_last_vis_df



In [4]:
ff_indices_of_each_cluster = cluster_analysis.find_alive_target_clusters(ff_real_position_sorted, ff_caught_T_new, ff_life_sorted, max_distance=50)

NameError: name 'ff_real_position_sorted' is not defined

# Basic data

In [2]:
## Retrieve monkey data
PLAYER = "monkey"
raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0330"
data_item = neural_vs_behavioral_class.NeuralVsBehavioralClass(raw_data_folder_path=raw_data_folder_path)
data_item.retrieve_or_make_monkey_data()
data_item.make_or_retrieve_ff_dataframe(exists_ok=True)
data_item.find_patterns()
data_item.make_PlotTrials_args()

monkey_information = data_item.monkey_information
ff_dataframe = data_item.ff_dataframe

ff_life_sorted = data_item.ff_life_sorted
ff_real_position_sorted = data_item.ff_real_position_sorted
ff_believed_position_sorted = data_item.ff_believed_position_sorted
cluster_around_target_indices = data_item.cluster_around_target_indices
ff_caught_T_new = data_item.ff_caught_T_new
caught_ff_num = len(ff_caught_T_new)
ff_flash_sorted = data_item.ff_flash_sorted
ff_flash_end_sorted = data_item.ff_flash_end_sorted
max_point_index = data_item.max_point_index
min_point_index = data_item.min_point_index




data_item.make_or_retrieve_target_clust_last_vis_df()
target_clust_last_vis_df = data_item.target_clust_last_vis_df
target_clust_last_vis_df['ff_index'] = target_clust_last_vis_df['target_index']


PlotTrials_args = (monkey_information, ff_dataframe, ff_life_sorted, ff_real_position_sorted, ff_believed_position_sorted, cluster_around_target_indices, ff_caught_T_new)


plot_polar_args = (monkey_information,
                    ff_dataframe, 
                    ff_life_sorted,
                    ff_real_position_sorted,
                    ff_caught_T_new,
                    ff_flash_sorted,)


trial_total_num = 2
PLAYER = "monkey"

Retrieved monkey_information
The number of points that were removed due to delta_position exceeding the ceiling is 0
Note: ff_caught_T_sorted is replaced with ff_caught_T_new
Retrieved ff_dataframe from all_monkey_data/processed_data/monkey_Bruno/data_0330/ff_dataframe.h5
When take out monkey subset for GUAT, 643 clusters out of 856 are too close to the target or the last target. Those clusters are filtered out.
The number of new trials that are used to separate stop clusters is 1338
Retrieved target_clust_last_vis_df


# retrieve right chunk of time

In [43]:
exists_ok = True
data_item.make_or_retrieve_target_clust_last_vis_df(exists_ok=exists_ok)
data_item.make_or_retrieve_target_last_vis_df(exists_ok=True)

data_item.target_clust_last_vis_df['nearby_vis_ff_indices'] = data_item.target_clust_last_vis_df['nearby_vis_ff_indices'].apply(
    lambda x: [int(i) for i in x.strip('[]').split(',') if i.strip().isdigit()])

data_item.target_clust_last_vis_df['num_nearby_vis_ff'] = data_item.target_clust_last_vis_df['nearby_vis_ff_indices'].apply(lambda x: len(x))

# add ff_caught_time and ff_caught_point_index
data_item.target_clust_last_vis_df['ff_caught_time'] = data_item.ff_caught_T_new[data_item.target_clust_last_vis_df['target_index'].values]
data_item.target_clust_last_vis_df['ff_caught_point_index'] = np.searchsorted(data_item.monkey_information['time'], data_item.target_clust_last_vis_df['ff_caught_time'].values)

Retrieved target_clust_last_vis_df
Retrieved target_last_vis_df


In [44]:
clust_sub = data_item.target_clust_last_vis_df[data_item.target_clust_last_vis_df['num_nearby_vis_ff'] == 1]
# print percentage of clust_sub
print("Percentage of targets not in a visible cluster out of all targets", len(clust_sub) / len(data_item.target_clust_last_vis_df) * 100)

Percentage of targets not in a visible cluster out of all targets 66.4424514200299


In [ ]:
# need 'last_vis_point_index' and 'ff_caught_point_index' and the point index in between


In [ ]:
# and then...let's see what data is used in neural and planning

In [ ]:
# incorpoate ff_caught_T_new converted to data points to clust_sub
# and then collect all the points between last visible time and caught time of each target
# eventually we would need information such as ff distance, angle, last visible time, etc....(might use information from neural_and_planning)
# also, look up how to analyze decoding from neural data.



In [46]:
clust_sub.columns

Index(['target_index', 'last_vis_point_index', 'last_vis_ff_index',
       'nearby_vis_ff_indices', 'time_since_last_vis', 'last_vis_dist',
       'last_vis_cum_dist', 'last_vis_ang', 'last_vis_ang_to_bndry',
       'last_vis_target_dist', 'last_vis_target_ang',
       'last_vis_target_ang_to_bndry', 'abs_last_vis_ang',
       'abs_last_vis_ang_to_bndry', 'abs_last_vis_target_ang',
       'abs_last_vis_target_ang_to_bndry', 'nearby_alive_ff_indices',
       'num_nearby_vis_ff', 'ff_caught_time', 'ff_caught_point_index'],
      dtype='object')

In [ ]:
# # Move 'new_column' to the front
# clust_sub.insert(0, 'new_column', clust_sub.pop('new_column'))